# 신경망 코드 학습


In [1]:
# 3계층의 신경망으로 MNIST 데이터를 학습하는 코드
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline
import random

# 신경망 클래스의 정의
class neuralNetwork:
    # 신경망 초기화하기
    def __init__(self,inputNodes,hiddenNodes,outputNodes,learningRate):
        # 입력, 은닉, 출력 계층의 노드 개수 설정
        self.inodes = inputNodes
        self.hnodes = hiddenNodes
        self.onodes = outputNodes
        
        # 학습률
        self.lr = learningRate
        
        # 가중치 행렬 wih와 who
        # 배열 내 가중치는 w_i_j로 표기. 노드 i에서 다음 계층의 노드 j로 연결됨을 의미
        # w11 w21
        # w21 w22 등
        self.wih=numpy.random.normal(0.0,pow(self.hnodes,-0.5),(self.hnodes,self.inodes))
        self.who=numpy.random.normal(0.0,pow(self.onodes,-0.5),(self.onodes,self.hnodes))
        
        # 활성화 함수로는 시그모이드 함수를 이용
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    # 신경망 학습시키기
    def train(self, input_list, target_list):
        #입력 리스트를 2차원의 행렬로 변환
        input=numpy.array(input_list,ndmin=2).T
        target=numpy.array(target_list,ndmin=2).T
        
        #은닉 계층으로 들어오는 신호를 계산
        hidden_inputs=numpy.dot(self.wih, input)
        #은닉 계층에서 나가는 신호를 계산
        hidden_outputs=self.activation_function(hidden_inputs)
        
        #최종 출력 계층으로 들어오는 신호를 계산
        final_inputs=numpy.dot(self.who,hidden_outputs)
        
        #최종 출력 계층에서 나가는 신호를 계산
        final_outputs=self.activation_function(final_inputs)
        
        # 오차는 (실제 값 - 계산 값)
        output_errors=target-final_outputs

        # 은닉 계층의 오차는 가중치에 의해 나뉜 출력 계층의 오차들을 재조합해 계산
        hidden_errors=numpy.dot(self.who.T, output_errors)
        
        # 은닉 계층과 출력 계층 간의 가중치 업데이트
        self.who += self.lr*numpy.dot((output_errors*final_outputs*(1.0-final_outputs)),numpy.transpose(hidden_outputs))
        
        # 입력 계층과 은닉 계층 간의 가중치 업데이트
        self.wih += self.lr*numpy.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)),numpy.transpose(input))
        
        pass

    # 신경망에 질의하기
    def query(self,input_list):
        # 입력 리스트를 2차원 행렬로 변환
        inputs = numpy.array(input_list, ndmin=2).T
        
        # 은닉 계층으로 들어오는 신호를 계산
        hidden_inputs=numpy.dot(self.wih,inputs)
        # 은닉 계층에서 나가는 신호를 계싼
        hidden_outputs=self.activation_function(hidden_inputs)
        # 최종 출력 계층으로 들어오는 신호를 계산
        final_inputs=numpy.dot(self.who,hidden_outputs)
        # 최종 출력 계층에서 나가는 신호를 계산
        final_outputs=self.activation_function(final_inputs)
        
        return final_outputs
        

In [3]:
# 입력, 은닉, 출력 노드의 수
input_nodes = 64 * 64  # 64x64 BMP 데이터를 입력으로 사용
hidden_nodes = 100  # 은닉층 노드 수
output_nodes = 154  # 타겟값의 종류는 154개

# 학습률
learning_rate = 0.3
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# font_data.csv 파일을 읽어 데이터 로드
csv_file = "../Data/font_data.csv"  # CSV 파일 경로
with open(csv_file, "r") as f:
    data_list = f.readlines()

# 데이터를 8:2로 나누기
random.shuffle(data_list)
split_index = int(len(data_list) * 0.8)
training_data_list = data_list[:split_index]
test_data_list = data_list[split_index:]

# 학습 데이터로 신경망 학습
for record in training_data_list:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99  # 타겟값 인덱스는 0부터 시작
    n.train(inputs, targets)

# 테스트 데이터로 신경망 성능 평가
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])  # 실제 정답
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    predicted_label = numpy.argmax(outputs)  # 예측값
    scorecard.append(1 if predicted_label == correct_label else 0)

# 성능 결과 출력
accuracy = sum(scorecard) / len(scorecard)
print(f"Performance: {accuracy * 100:.2f}%")

# 테스트: 임의의 데이터로 글자 예측
test_sample = test_data_list[0].split(',')
test_inputs = (numpy.asfarray(test_sample[1:]) / 255.0 * 0.99) + 0.01
test_output = n.query(test_inputs)
predicted_character_index = numpy.argmax(test_output)
print(f"Predicted Target Index: {predicted_character_index}")

Performance: 0.00%
Predicted Target Index: 59
